# 1. Cleaning the original raw action units data 

In this notebook, the files we got from the FAINT project team is looked at, cleaned and made ready for preprocessing in TS Fresh. 

In [10]:
# import 
import zipfile
import os
import pandas as pd
import csv
import socket  
import pickle
import re

# need to enable iterative imputer explicitly since its still experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Unzipping

In [2]:
# Define paths to the zipped and unzipped folders 
zip_file_path = '/Users/dionnespaltman/Desktop/V4/full donation data.zip'
output_folder = '/Users/dionnespaltman/Desktop/V4/full donation data - unzipped'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Unzip the folder
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_folder)

print("Unzipped folder created at:", output_folder)

Unzipped folder created at: /Users/dionnespaltman/Desktop/V4/full donation data - unzipped


Check the size of the folder. It's 18684.90839099884 MB. 

In [6]:
# Define the path to the folder
folder_path_unzipped_data = '/Users/dionnespaltman/Desktop/V4/full donation data - unzipped/full donation data'

# Initialize total size variable
total_size = 0

# Iterate over all files in the folder
for filename in os.listdir(folder_path_unzipped_data):
    file_path = os.path.join(folder_path_unzipped_data, filename)
    # Check if it's a file (not a directory)
    if os.path.isfile(file_path):
        # Get the size of the file and add it to total_size
        total_size += os.path.getsize(file_path)

# Convert total size to a human-readable format (e.g., bytes to megabytes)
total_size_mb = total_size / (1024 * 1024)  # Convert bytes to megabytes

print("Total size of all files in the folder:", total_size_mb, "MB")

Total size of all files in the folder: 18684.90839099884 MB


# Creating a dictionary with all the files 

I want to access the file in a easy way. Since the files have important information in their filename, I will put all the files in a pandas dictionary (very efficient in my experience). The filename will serve as a key. In total 403 files were uploaded in the dictionary, that is correct. 

In [ ]:
# List all files in the folder
file_names = os.listdir(folder_path_unzipped_data)

# Read a subset of files into a dictionary
data = {}
num_files_to_read = 450  # Adjust the number of files to read as needed
for i, file_name in enumerate(file_names):
    if i >= num_files_to_read:
        break
    if file_name.endswith('.csv'):  # Assuming the files are CSV format
        file_path = os.path.join(folder_path_unzipped_data, file_name)
        try:
            data[file_name] = pd.read_csv(file_path)
            print("File loaded successfully:", file_name)
        except Exception as e:
            print("Error loading file:", file_name, "- Error:", e)

# Now you have a dictionary 'data' containing DataFrames for each file (up to the specified number)
# Access them using keys (file names)
# For example:
print("Number of files loaded:", len(data))

Double check if the dicitionary is not empty. Check the structure. 

In [4]:
# Check if the dictionary is not empty
if data:
    # Get the first key (file name) and its corresponding DataFrame
    first_file_name = next(iter(data.keys()))
    first_df = data[first_file_name]
    
    # Display the DataFrame
    print("DataFrame for the first file '{}' in the dictionary:".format(first_file_name))
    display(first_df.head(10))
else:
    print("The dictionary is empty. No files loaded.")

DataFrame for the first file '16-07.csv' in the dictionary:


,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.00,0.98,1,-0.186231,-0.071084,-0.979931,-0.321020,-0.025671,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2,0,0.04,0.98,1,-0.195272,-0.066113,-0.978518,-0.321741,-0.022495,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3,0,0.08,0.98,1,-0.183891,-0.072790,-0.980248,-0.325824,-0.032522,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,0,0.12,0.98,1,-0.189050,-0.061480,-0.980041,-0.323833,-0.018319,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5,0,0.16,0.98,1,-0.193372,-0.059505,-0.979319,-0.317909,-0.020461,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,6,0,0.20,0.98,1,-0.182038,-0.053976,-0.981809,-0.320776,-0.025344,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,7,0,0.24,0.98,1,-0.174060,-0.062096,-0.982775,-0.332763,-0.027208,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,8,0,0.28,0.98,1,-0.183968,-0.068336,-0.980554,-0.325630,-0.040619,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,9,0,0.32,0.98,1,-0.172816,-0.076083,-0.982011,-0.330309,-0.047691,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,10,0,0.36,0.98,1,-0.162287,-0.070443,-0.984226,-0.325321,-0.033416,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Saving the dictionary with all files as a pickle file. 

In [5]:
# Specify the file path to save the dictionary
save_path = '/Users/dionnespaltman/Desktop/V4/action_units/dictionary.pkl'

# Save the dictionary to a file using pickle
with open(save_path, 'wb') as file:
    pickle.dump(data, file)

print("Dictionary saved to:", save_path)

Dictionary saved to: /Users/dionnespaltman/Desktop/V3/data_dictionary_temp.pkl


# Loading the dictionary 

If you've already done the previous steps, then you can go immediately to this step and load the dictionary. 

In [ ]:
# Specify the file path from which to load the dictionary
load_path = '/Users/dionnespaltman/Desktop/V4/action_units/dictionary.pkl'  

# Load the dictionary from the file using pickle
with open(load_path, 'rb') as file:
    data = pickle.load(file)

print("Dictionary loaded from:", data)

# Dropping columns 

In [ ]:
# Check if the dictionary is not empty
if data:
    # Get the first key (file name) and its corresponding DataFrame
    first_file_name = next(iter(data.keys()))
    first_df = data[first_file_name]
    
    # Print the column names of the DataFrame
    print("Column names of the first file '{}' in the dictionary:".format(first_file_name))
    print(first_df.columns.tolist())
else:
    print("The dictionary is empty. No files loaded.")

I want to check one single dictionary and use a key to see if everything works correctly as it should be. The shape of the dataframe is (2750, 714). It has 2750 rows and 714 columns. A lot of the rows are not interesting for the current research. 

In [78]:
# Get the DataFrame with the key '16-07.csv'
df_16_07 = data['16-07.csv']

# Get the shape of the DataFrame
shape_16_07 = df_16_07.shape

# Print the shape
print("Shape of the DataFrame with key '16-07.csv':", shape_16_07)

Shape of the DataFrame with key '16-07.csv': (2750, 714)


Here I'm making a function to be able to delete the columns I'm not interested in. These include columns with the following substrings: 'gaze', 'p', 'x', 'X', 'Y', 'y', 'Z',  'pose', 'eye'. After I've made the function, I test it using some examples. 

In [79]:
def get_columns_to_delete(columns):
    # List of substrings to search for in column names
    substrings = ['gaze', 'p', 'x', 'X', 'Y', 'y', 'Z',  'pose', 'eye']

    # Initialize an empty list to store column names to delete
    columns_to_delete = []

    # Iterate through each column name
    for column in columns:
        # Check if any of the substrings are present in the column name
        if any(sub in column for sub in substrings):
            # If present, add the column name to the list of columns to delete
            columns_to_delete.append(column)

    return columns_to_delete

In [80]:
# Example usage:
columns = ['AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'gaze_0_x', 'gaze_0_y', 'pose_Rx', 'pose_Ry']
columns_to_delete = get_columns_to_delete(columns)
print(columns_to_delete)

['gaze_0_x', 'gaze_0_y', 'pose_Rx', 'pose_Ry']


Then I test it on a seperate dataframe in the dictionary. From the 714 columsn in total, there are 675 columns that can be deleted. 

In [81]:
# Get the DataFrame from the dictionary
df_to_process = data['16-07.csv']

# Get the list of column names
columns = df_to_process.columns
print(len(columns))

# Get the list of columns to delete
columns_to_delete = get_columns_to_delete(columns)

# Print the list of columns to delete
print(len(columns_to_delete))

714
675


The new, cleaner version of the dictionary is called data_dropped. I iterate through all the keys and dataframes in the old dictionary data. 

In [82]:
# Define the new dictionary to store modified DataFrames
data_dropped = {}

# Iterate through all the DataFrames in the original dictionary
for key, df in data.items():
    # Get the list of column names for the current DataFrame
    columns = df.columns
    
    # Get the list of columns to delete
    columns_to_delete = get_columns_to_delete(columns)
    
    # Create a new DataFrame without the columns to delete
    df_dropped = df.drop(columns=columns_to_delete)
    
    # Add the new DataFrame to the new dictionary
    data_dropped[key] = df_dropped

In [83]:
# Specify the file path to save the dictionary
save_path = '/Users/dionnespaltman/Desktop/V4/action_units/dictionary_dropped.pkl'

# Save the dictionary to a file using pickle
with open(save_path, 'wb') as file:
    pickle.dump(data_dropped, file)

print("Dictionary saved to:", save_path)

Dictionary saved to: /Users/dionnespaltman/Desktop/V4/action_units/dictionary_dropped.pkl


New shape of a single dataframe is (2750, 39). 

In [84]:
# Get the DataFrame with the key '16-07.csv'
df_16_07 = data_dropped['16-07.csv']

# Get the shape of the DataFrame
shape_16_07 = df_16_07.shape

# Print the shape
print("Shape of the DataFrame with key '16-07.csv':", shape_16_07)

# Print column names
print(df_16_07.columns)

Shape of the DataFrame with key '16-07.csv': (2750, 39)
Index(['frame', ' face_id', ' confidence', ' success', ' AU01_r', ' AU02_r',
       ' AU04_r', ' AU05_r', ' AU06_r', ' AU07_r', ' AU09_r', ' AU10_r',
       ' AU12_r', ' AU14_r', ' AU15_r', ' AU17_r', ' AU20_r', ' AU23_r',
       ' AU25_r', ' AU26_r', ' AU45_r', ' AU01_c', ' AU02_c', ' AU04_c',
       ' AU05_c', ' AU06_c', ' AU07_c', ' AU09_c', ' AU10_c', ' AU12_c',
       ' AU14_c', ' AU15_c', ' AU17_c', ' AU20_c', ' AU23_c', ' AU25_c',
       ' AU26_c', ' AU28_c', ' AU45_c'],
      dtype='object')


# Loading the previous dataframe 

In [14]:
# Specify the file path where the pickle file is saved
save_path = '/Users/dionnespaltman/Desktop/V4/action_units/dictionary_dropped.pkl'

# Load the dictionary from the pickle file
with open(save_path, 'rb') as file:
    data = pickle.load(file)

# Display the loaded data
print("Dictionary loaded successfully.")

Dictionary loaded successfully.


# Adding ID 

It is important to be able to easily access the ID. Here I create a new column in each dataframe and store the ID there. 

In [15]:
# Iterate over the keys of the dictionary
for key in data.keys():
    # Extract the ID from the key
    id = re.search(r'\d+', key).group()
    
    # Add a new column to the dataframe with the extracted ID
    data[key]['ID'] = id

Check if it's correctly implemented. 

In [16]:
# Get the DataFrame from the dictionary
df_16_07 = data['16-07.csv']

display(df_16_07)

,frame,face_id,confidence,success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,ID
0,1,0,0.98,1,0.00,0.00,0.92,0.0,0.23,1.15,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16
1,2,0,0.98,1,0.07,0.00,0.78,0.0,0.16,0.95,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16
2,3,0,0.98,1,0.15,0.00,0.81,0.0,0.16,0.91,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,16
3,4,0,0.98,1,0.14,0.10,0.82,0.0,0.15,1.01,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16
4,5,0,0.98,1,0.15,0.11,0.87,0.0,0.15,0.99,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,2746,0,0.98,1,0.17,0.10,0.54,0.0,0.01,1.33,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16
2746,2747,0,0.98,1,0.22,0.13,0.57,0.0,0.01,1.29,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16
2747,2748,0,0.98,1,0.14,0.03,0.61,0.0,0.00,1.24,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16
2748,2749,0,0.98,1,0.09,0.03,0.60,0.0,0.00,1.06,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16


# Adding Stage

Each file has the information of one or more stages. The stages are the different stages of the entire donation proccess. 

- Stage 1: Recorded a one-minute video. Initial VVR measurement was taken.
- Stage 2: Donors completed standard registration forms at Sanquin. Participants were recorded as they progressed through the blood donation procedure. Continued to take VVR measurements.
- Stage 3: Donors either waited in the waiting area or were sent directly to the donation chair, depending on BCC location. Recorded a third video, lasting 1 to 2 minutes, for those waiting. Donors self-reported VVR during this time.
- Stages 4–6: Continuous video recording in the donation chair. VVR levels assessed three times: at needle insertion (stage 4), around 300 mL of donated blood (stage 5), and during needle uncoupling (stage 6).
- Stage 7: Final recording and VVR level assessment in the waiting area. Donors recuperated from the donation process.

Unfortunately the formatting is not the same for each file. So it was necessary to do some testing. 

In [17]:
def extract_timeframe(filename):
    # Extract numerical values from the filename
    numerical_values = re.findall(r'\d+', filename)
    
    # Convert each numerical value to an integer
    stages = [int(value) for value in numerical_values]
    
    return stages

In [18]:
# Test cases
filename1 = '16-07.csv'
filename2 = '6-03,04,05,06.csv'

stages_file1 = extract_timeframe(filename1)
stages_file2 = extract_timeframe(filename2)

print("Timeframes from filename 1:", stages_file1)
print("Timeframes from filename 2:", stages_file2)

Timeframes from filename 1: [16, 7]
Timeframes from filename 2: [6, 3, 4, 5, 6]


In [ ]:
# Check to see if all the keys have the same format 
def check_key_format(keys):
    # Regular expression patterns to match the two formats
    pattern_1 = re.compile(r'^\d+_\d+(\,\d+)*\.csv$')
    pattern_2 = re.compile(r'^\d+(-\d+(\,\d+)*)*\.csv$')

    # Lists to store keys with different formats
    format_1_keys = []
    format_2_keys = []
    other_keys = []

    # Iterate through all keys
    for key in keys:
        # Match the key format with both patterns
        match_1 = pattern_1.match(key)
        match_2 = pattern_2.match(key)
        if match_1:
            format_1_keys.append(key)
        elif match_2:
            format_2_keys.append(key)
        else:
            other_keys.append(key)

    return format_1_keys, format_2_keys, other_keys

# Usage example:
keys = data.keys()  
format_1_keys, format_2_keys, other_keys = check_key_format(keys)
print("Keys with format 1:", format_1_keys)
print("Keys with format 2:", format_2_keys)
print("Other keys:", other_keys)


There are a few keys with a different format: ['105_04 donation not completed. No blood flow.csv', 'DSCN2370.csv', '126_01 (5).csv', '126_01 (4).csv', '126_01 (3).csv']. 

Since it seems these are not correct or double, we delete these from the data. 

In [20]:
# Filter out dataframes with keys in other_keys
data_filtered = {key: value for key, value in data.items() if key not in other_keys}

# Verify that dataframes with keys in other_keys are deleted
print(len(data_filtered))

398


# Modify keys

We used to have 403 files (all were put into a dictionary). Then 5 dataframes in the dictionary were dropped, so we have a length of 398. 

In [21]:
# Iterate through the keys and modify keys to the desired format
for key in list(data_filtered.keys()):
    parts = key.split('-')
    if len(parts) >= 2 and parts[-1].endswith('.csv'):
        id_part = parts[0]
        stage_part = ",".join(parts[1].split(','))
        new_key = f"{id_part}_{stage_part}.csv"
        data_filtered[new_key] = data_filtered.pop(key)

# Verify the modified keys
# print("Modified keys in desired format:", data_filtered.keys())
print(len(data_filtered))

398


Create a column in the dataframes for stage. 

In [24]:
# Iterate through the dataframes in the data_filtered dictionary
for key, df in data_filtered.items():
    # Remove the '.csv' suffix from the key
    key = key.replace('.csv', '')
    # Extract the timepoints from the key
    stages_str = key.split('_')[1]
    # Remove leading zeros from the timepoints string
    stages_str = ','.join(str(int(tp.lstrip('0'))) for tp in stages_str.split(','))
    # Convert the timepoints string into a list of integers
    stages_list = [int(tp) for tp in stages_str.split(',')]
    # Add the Timeframe column to the dataframe
    df['Stage'] = [stages_list] * len(df)

# Get the DataFrame from the dictionary
df_95 = data_filtered['95_04,05,06.csv']

display(df_95)

,frame,face_id,confidence,success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,ID,Stage
0,1,0.0,0.98,1.0,0.00,0.00,0.00,0.0,0.43,0.00,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,95,"[4, 5, 6]"
1,2,0.0,0.98,1.0,0.00,0.00,0.00,0.0,0.41,0.00,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,95,"[4, 5, 6]"
2,3,0.0,0.98,1.0,0.00,0.00,0.00,0.0,0.37,0.00,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,95,"[4, 5, 6]"
3,4,0.0,0.98,1.0,0.00,0.00,0.00,0.0,0.29,0.00,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,95,"[4, 5, 6]"
4,5,0.0,0.98,1.0,0.00,0.00,0.00,0.0,0.18,0.00,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,95,"[4, 5, 6]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17880,17881,0.0,0.98,1.0,2.59,0.59,0.58,0.0,0.00,0.16,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,95,"[4, 5, 6]"
17881,17882,0.0,0.98,1.0,2.73,0.81,0.65,0.0,0.00,0.26,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,95,"[4, 5, 6]"
17882,17883,0.0,0.98,1.0,2.73,0.79,0.55,0.0,0.00,0.19,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,95,"[4, 5, 6]"
17883,17884,0.0,0.98,1.0,2.77,0.82,0.44,0.0,0.00,0.32,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,95,"[4, 5, 6]"


# Merge

To be able to work sufficiently and preprocess using TS Fresh, all the dataframes in the dicitonary should be merged into one. 
The new big dataframe is just called df. 

In [26]:
df = pd.concat(data_filtered.values(), ignore_index=True)

# Rename columns using a dictionary where keys are the current column names and values are the new column names
df = df.rename(columns={'frame': 'Frame',
                        ' face_id': 'Face_id', 
                        ' confidence': 'Confidence', 
                        ' success': 'Success'
                        })

display(df)

,Frame,Face_id,Confidence,Success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,ID,Stage
0,1,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
1,2,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
2,3,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
3,4,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
4,5,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250413,2196,0.0,0.98,1.0,0.62,0.36,0.30,0.0,0.70,0.47,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3250414,2197,0.0,0.98,1.0,0.63,0.38,0.25,0.0,0.78,0.56,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3250415,2198,0.0,0.98,1.0,0.69,0.51,0.26,0.0,0.79,0.56,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3250416,2199,0.0,0.98,1.0,0.67,0.46,0.32,0.0,0.81,0.56,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]


# Drop columns with confidence below 0.80

If values are not confident enough, they should be dropped. We go from 3250418 to 3174403. 

In [27]:
print(df.shape)
df = df[df['Confidence'] >= 0.80]
print(df.shape)

(3250418, 41)
(3174403, 41)


# Check for missing values

In [28]:
# Check for missing values in 'ID' and 'Timeframe' columns
missing_id = df['ID'].isna().any()
missing_stage = df['Stage'].isna().any()

# Print the results
print("Missing values in 'ID' column:", missing_id)
print("Missing values in 'Timeframe' column:", missing_stage)

Missing values in 'ID' column: False
Missing values in 'Timeframe' column: False


# Check column names

The column names should be consistent, so no spaces at the beginning and all starting with a capital letter. 

In [103]:
# Print the column names
print("Column names:", df.columns.tolist())
print(len(df.columns.tolist()))

Column names: ['Frame', 'Face_id', 'Confidence', 'Success', ' AU01_r', ' AU02_r', ' AU04_r', ' AU05_r', ' AU06_r', ' AU07_r', ' AU09_r', ' AU10_r', ' AU12_r', ' AU14_r', ' AU15_r', ' AU17_r', ' AU20_r', ' AU23_r', ' AU25_r', ' AU26_r', ' AU45_r', ' AU01_c', ' AU02_c', ' AU04_c', ' AU05_c', ' AU06_c', ' AU07_c', ' AU09_c', ' AU10_c', ' AU12_c', ' AU14_c', ' AU15_c', ' AU17_c', ' AU20_c', ' AU23_c', ' AU25_c', ' AU26_c', ' AU28_c', ' AU45_c', 'ID', 'Stage']
41


In [33]:
# Rename columns using a dictionary where keys are the current column names and values are the new column names
df = df.rename(columns={' AU01_r': 'AU01_r',
                        ' AU02_r': 'AU02_r',
                        ' AU04_r': 'AU04_r',
                        ' AU05_r': 'AU05_r',
                        ' AU06_r': 'AU06_r',
                        ' AU07_r': 'AU07_r',
                        ' AU09_r': 'AU09_r',
                        ' AU10_r': 'AU10_r',
                        ' AU12_r': 'AU12_r',
                        ' AU14_r': 'AU14_r',
                        ' AU15_r': 'AU15_r',
                        ' AU17_r': 'AU17_r',
                        ' AU20_r': 'AU20_r',
                        ' AU23_r': 'AU23_r',
                        ' AU25_r': 'AU25_r',
                        ' AU26_r': 'AU26_r',
                        ' AU45_r': 'AU45_r',
                        ' AU01_c': 'AU01_c',
                        ' AU02_c': 'AU02_c',
                        ' AU04_c': 'AU04_c',
                        ' AU05_c': 'AU05_c',
                        ' AU06_c': 'AU06_c',
                        ' AU07_c': 'AU07_c',
                        ' AU09_c': 'AU09_c',
                        ' AU10_c': 'AU10_c',
                        ' AU12_c': 'AU12_c',
                        ' AU14_c': 'AU14_c',
                        ' AU15_c': 'AU15_c',
                        ' AU17_c': 'AU17_c',
                        ' AU20_c': 'AU20_c',
                        ' AU23_c': 'AU23_c',
                        ' AU25_c': 'AU25_c',
                        ' AU26_c': 'AU26_c',
                        ' AU28_c': 'AU28_c',
                        ' AU45_c': 'AU45_c'
                        })

In [34]:
# Print the column names
print("Column names:", df.columns.tolist())
print(len(df.columns.tolist()))

Column names: ['Frame', 'Face_id', 'Confidence', 'Success', 'AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', 'AU45_r', 'AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c', 'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c', 'AU28_c', 'AU45_c', 'ID', 'Stage']
41


# Check number of unique IDs

Unfortunately, the data I received is not complete. We only have data from 184 IDs. However, we don't have information from all stages for every one of those IDs. So the data is very limited. 

In [35]:
# Get number of unique IDs
num_unique_ids = len(df['ID'].unique())

print("Number of Unique IDs:", num_unique_ids)

Number of Unique IDs: 184


# Save file 

The file we end up with has 3174403 rows and 41 columns. 

In [36]:
print(df.columns)

Index(['Frame', 'Face_id', 'Confidence', 'Success', 'AU01_r', 'AU02_r',
       'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r',
       'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r',
       'AU45_r', 'AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c',
       'AU09_c', 'AU10_c', 'AU12_c', 'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c',
       'AU23_c', 'AU25_c', 'AU26_c', 'AU28_c', 'AU45_c', 'ID', 'Stage'],
      dtype='object')


In [37]:
display(df)

,Frame,Face_id,Confidence,Success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,ID,Stage
103,104,0.0,0.88,1.0,0.00,0.00,0.00,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
104,105,0.0,0.98,1.0,0.00,0.00,0.00,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
111,112,0.0,0.98,1.0,0.31,0.59,0.00,0.0,0.31,0.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
114,115,0.0,0.98,1.0,0.00,0.12,0.00,0.0,0.32,0.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
153,154,0.0,0.98,1.0,1.26,1.67,0.00,0.0,0.39,0.26,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250413,2196,0.0,0.98,1.0,0.62,0.36,0.30,0.0,0.70,0.47,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3250414,2197,0.0,0.98,1.0,0.63,0.38,0.25,0.0,0.78,0.56,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3250415,2198,0.0,0.98,1.0,0.69,0.51,0.26,0.0,0.79,0.56,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3250416,2199,0.0,0.98,1.0,0.67,0.46,0.32,0.0,0.81,0.56,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]


In [38]:
df.to_csv('/Users/dionnespaltman/Desktop/V5/clean_actionunits.csv', sep=',')

# Loading df 

In [4]:
df = pd.read_csv('/Users/dionnespaltman/Desktop/V5/clean_actionunits.csv', sep=',')

if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)

display(df)

,Frame,Face_id,Confidence,Success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,ID,Stage
0,104.0,0.0,0.88,1.0,0.00,0.00,0.00,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
1,105.0,0.0,0.98,1.0,0.00,0.00,0.00,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
2,112.0,0.0,0.98,1.0,0.31,0.59,0.00,0.0,0.31,0.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
3,115.0,0.0,0.98,1.0,0.00,0.12,0.00,0.0,0.32,0.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
4,154.0,0.0,0.98,1.0,1.26,1.67,0.00,0.0,0.39,0.26,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3174398,2196.0,0.0,0.98,1.0,0.62,0.36,0.30,0.0,0.70,0.47,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3174399,2197.0,0.0,0.98,1.0,0.63,0.38,0.25,0.0,0.78,0.56,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3174400,2198.0,0.0,0.98,1.0,0.69,0.51,0.26,0.0,0.79,0.56,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3174401,2199.0,0.0,0.98,1.0,0.67,0.46,0.32,0.0,0.81,0.56,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]


In [5]:
print(len(df['ID'].unique()))

184


Here I check information from what stages is available. At first, I wanted to use the recordings from the waiting room to add them as a feature. However, this is not possible, since there is so little data from these timepoints. So I am only going to use data from stage 4, 5 and 6 which is when the participants are in the donation chair. 

In [6]:
# Count occurrences of each value in the "Stage" column
stage_counts = df['Stage'].value_counts()

print(stage_counts)

[4, 5, 6]       2118735
[3, 4, 5, 6]     285556
[7]              269554
[1]              163222
[3]              134364
[2]               78159
[4]               68762
[4, 5]            40338
[4, 6]            15713
Name: Stage, dtype: int64


# Missing values 

There are quite some missing values. This is possible when participants look away from the camera. Then naturally OpenFace wasn't able to get any information from their faces. 

In [7]:
nan_counts = df.notna().sum()
print(nan_counts)

Frame         3174403
Face_id       3174403
Confidence    3174403
Success       3174403
AU01_r        2931009
AU02_r        2931009
AU04_r        2931009
AU05_r        2931009
AU06_r        2931009
AU07_r        2931009
AU09_r        2931009
AU10_r        2931009
AU12_r        2931009
AU14_r        2931009
AU15_r        2931009
AU17_r        2931009
AU20_r        2931009
AU23_r        2931009
AU25_r        2931009
AU26_r        2931009
AU45_r        2931009
AU01_c        2931009
AU02_c        2931009
AU04_c        2931009
AU05_c        2931009
AU06_c        2931009
AU07_c        2931009
AU09_c        2931009
AU10_c        2931009
AU12_c        2931009
AU14_c        2931009
AU15_c        2931009
AU17_c        2931009
AU20_c        2931009
AU23_c        2931009
AU25_c        2931009
AU26_c        2931009
AU28_c        2931009
AU45_c        2931009
ID            3174403
Stage         3174403
dtype: int64


In [8]:
nan_counts_au_df = df.isna().sum()
print(nan_counts_au_df)

Frame              0
Face_id            0
Confidence         0
Success            0
AU01_r        243394
AU02_r        243394
AU04_r        243394
AU05_r        243394
AU06_r        243394
AU07_r        243394
AU09_r        243394
AU10_r        243394
AU12_r        243394
AU14_r        243394
AU15_r        243394
AU17_r        243394
AU20_r        243394
AU23_r        243394
AU25_r        243394
AU26_r        243394
AU45_r        243394
AU01_c        243394
AU02_c        243394
AU04_c        243394
AU05_c        243394
AU06_c        243394
AU07_c        243394
AU09_c        243394
AU10_c        243394
AU12_c        243394
AU14_c        243394
AU15_c        243394
AU17_c        243394
AU20_c        243394
AU23_c        243394
AU25_c        243394
AU26_c        243394
AU28_c        243394
AU45_c        243394
ID                 0
Stage              0
dtype: int64


In [9]:
print(243394/3174403*100)

7.667394467558151


# Dealing with missing values - MICE

Because we're missing about 7 percent of the action units, it would not be smart to simply delete these rows. We will use MICE to deal with the missing values. 
Link: https://www.machinelearningplus.com/machine-learning/mice-imputation/?utm_content=cmp-true 

In [11]:
# Define imputer
imputer = IterativeImputer(random_state=100, max_iter=10)

In [12]:
# Use Numeric Features
data = df.loc[:, ['AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', 'AU45_r', 'AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c', 'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c', 'AU28_c', 'AU45_c']]
data

,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,0.00,0.00,0.00,0.0,1.34,0.22,0.57,1.56,1.52,0.54,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00,0.00,0.00,0.0,1.34,0.22,0.57,1.56,1.52,0.54,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.31,0.59,0.00,0.0,0.31,0.31,0.23,0.21,0.62,0.12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.12,0.00,0.0,0.32,0.06,0.10,0.60,0.50,0.29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.26,1.67,0.00,0.0,0.39,0.26,0.00,1.22,0.64,0.18,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3174398,0.62,0.36,0.30,0.0,0.70,0.47,0.00,0.06,1.54,1.10,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3174399,0.63,0.38,0.25,0.0,0.78,0.56,0.00,0.04,1.45,1.07,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3174400,0.69,0.51,0.26,0.0,0.79,0.56,0.00,0.00,1.43,1.04,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3174401,0.67,0.46,0.32,0.0,0.81,0.56,0.00,0.02,1.43,1.16,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


Running everything up until the imputer was very fast. Running the imputer took around 6 minutes. 

In [13]:
# fit on the dataset
imputer.fit(data)

IterativeImputer(random_state=100)

In [14]:
action_units_imputed = imputer.transform(data)

In [17]:
# Replace with imputed values
df.loc[:, ['AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 
                     'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', 'AU45_r', 'AU01_c', 
                     'AU02_c', 'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c', 'AU14_c', 
                     'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c', 'AU28_c', 'AU45_c']] = action_units_imputed
df

,Frame,Face_id,Confidence,Success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,ID,Stage
0,104.0,0.0,0.88,1.0,0.00,0.00,0.00,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
1,105.0,0.0,0.98,1.0,0.00,0.00,0.00,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
2,112.0,0.0,0.98,1.0,0.31,0.59,0.00,0.0,0.31,0.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
3,115.0,0.0,0.98,1.0,0.00,0.12,0.00,0.0,0.32,0.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
4,154.0,0.0,0.98,1.0,1.26,1.67,0.00,0.0,0.39,0.26,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3174398,2196.0,0.0,0.98,1.0,0.62,0.36,0.30,0.0,0.70,0.47,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3174399,2197.0,0.0,0.98,1.0,0.63,0.38,0.25,0.0,0.78,0.56,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3174400,2198.0,0.0,0.98,1.0,0.69,0.51,0.26,0.0,0.79,0.56,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3174401,2199.0,0.0,0.98,1.0,0.67,0.46,0.32,0.0,0.81,0.56,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]


In [19]:
num_unique_ids = df['ID'].nunique()

print(num_unique_ids)

184


Then we will double check if everything went correctly. We should not have any missing values anymore. This is important to be able to implement TS Fresh. 

In [21]:
nan_counts_au_df = df.isna().sum()
print(nan_counts_au_df)

Frame         0
Face_id       0
Confidence    0
Success       0
AU01_r        0
AU02_r        0
AU04_r        0
AU05_r        0
AU06_r        0
AU07_r        0
AU09_r        0
AU10_r        0
AU12_r        0
AU14_r        0
AU15_r        0
AU17_r        0
AU20_r        0
AU23_r        0
AU25_r        0
AU26_r        0
AU45_r        0
AU01_c        0
AU02_c        0
AU04_c        0
AU05_c        0
AU06_c        0
AU07_c        0
AU09_c        0
AU10_c        0
AU12_c        0
AU14_c        0
AU15_c        0
AU17_c        0
AU20_c        0
AU23_c        0
AU25_c        0
AU26_c        0
AU28_c        0
AU45_c        0
ID            0
Stage         0
dtype: int64


# Saving the action units file with the imputed values 

In [ ]:
df.to_csv("/Users/dionnespaltman/Desktop/V5/imputed_data.csv", sep=',')

# Loading action units file with the imputed values 

In [ ]:
df = pd.read_csv("/Users/dionnespaltman/Desktop/V5/imputed_data.csv")

display(df)

,Unnamed: 0,Frame,Face_id,Confidence,Success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,ID,Stage
0,0,104.0,0.0,0.88,1.0,0.00,0.00,0.00,0.0,1.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
1,1,105.0,0.0,0.98,1.0,0.00,0.00,0.00,0.0,1.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
2,2,112.0,0.0,0.98,1.0,0.31,0.59,0.00,0.0,0.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
3,3,115.0,0.0,0.98,1.0,0.00,0.12,0.00,0.0,0.32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
4,4,154.0,0.0,0.98,1.0,1.26,1.67,0.00,0.0,0.39,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3174398,3174398,2196.0,0.0,0.98,1.0,0.62,0.36,0.30,0.0,0.70,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3174399,3174399,2197.0,0.0,0.98,1.0,0.63,0.38,0.25,0.0,0.78,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3174400,3174400,2198.0,0.0,0.98,1.0,0.69,0.51,0.26,0.0,0.79,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]
3174401,3174401,2199.0,0.0,0.98,1.0,0.67,0.46,0.32,0.0,0.81,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7]


# Creating file with just Stage = [4, 5, 6]

In [23]:
# Count occurrences of Stage values in action_units DataFrame
stage_counts = df['Stage'].value_counts()
print(stage_counts)

[4, 5, 6]       2118735
[3, 4, 5, 6]     285556
[7]              269554
[1]              163222
[3]              134364
[2]               78159
[4]               68762
[4, 5]            40338
[4, 6]            15713
Name: Stage, dtype: int64


In [24]:
# Convert the 'Stage' column to a string representation
df['Stage_str'] = df['Stage'].apply(lambda x: str(x))

# Display the DataFrame to check the conversion
print("DataFrame with 'Stage' as string:")
display(df)

# Filter rows where 'Stage_str' is exactly "[4, 5, 6]"
filtered_df = df[df['Stage_str'] == str([4, 5, 6])]

# Display the filtered DataFrame
print("Filtered DataFrame where Stage is exactly '[4, 5, 6]':")
display(filtered_df)

DataFrame with 'Stage' as string:


,Frame,Face_id,Confidence,Success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,ID,Stage,Stage_str
0,104.0,0.0,0.88,1.0,0.00,0.00,0.00,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
1,105.0,0.0,0.98,1.0,0.00,0.00,0.00,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
2,112.0,0.0,0.98,1.0,0.31,0.59,0.00,0.0,0.31,0.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
3,115.0,0.0,0.98,1.0,0.00,0.12,0.00,0.0,0.32,0.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
4,154.0,0.0,0.98,1.0,1.26,1.67,0.00,0.0,0.39,0.26,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3174398,2196.0,0.0,0.98,1.0,0.62,0.36,0.30,0.0,0.70,0.47,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7],[7]
3174399,2197.0,0.0,0.98,1.0,0.63,0.38,0.25,0.0,0.78,0.56,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7],[7]
3174400,2198.0,0.0,0.98,1.0,0.69,0.51,0.26,0.0,0.79,0.56,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7],[7]
3174401,2199.0,0.0,0.98,1.0,0.67,0.46,0.32,0.0,0.81,0.56,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,25,[7],[7]


Filtered DataFrame where Stage is exactly '[4, 5, 6]':


,Frame,Face_id,Confidence,Success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,ID,Stage,Stage_str
0,104.0,0.0,0.88,1.0,0.00,0.00,0.00,0.00,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
1,105.0,0.0,0.98,1.0,0.00,0.00,0.00,0.00,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
2,112.0,0.0,0.98,1.0,0.31,0.59,0.00,0.00,0.31,0.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
3,115.0,0.0,0.98,1.0,0.00,0.12,0.00,0.00,0.32,0.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
4,154.0,0.0,0.98,1.0,1.26,1.67,0.00,0.00,0.39,0.26,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3106523,21069.0,0.0,0.93,1.0,2.08,2.22,0.89,0.16,0.22,0.47,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,27,"[4, 5, 6]","[4, 5, 6]"
3106524,21070.0,0.0,0.93,1.0,2.03,2.14,0.87,0.69,0.27,0.85,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,27,"[4, 5, 6]","[4, 5, 6]"
3106525,21071.0,0.0,0.93,1.0,2.02,2.11,1.12,0.49,0.47,0.67,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,27,"[4, 5, 6]","[4, 5, 6]"
3106526,21072.0,0.0,0.93,1.0,2.43,2.01,1.11,0.65,0.35,0.82,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,27,"[4, 5, 6]","[4, 5, 6]"


Okay, so we had data from 184 participants. But we only have information from stages 4, 5 and 6 from 104 IDs. 

In [25]:
# Count the unique IDs in the filtered DataFrame
num_unique_ids = filtered_df['ID'].nunique()
print("Number of unique IDs in filtered DataFrame:", num_unique_ids)

print(filtered_df.shape)

Number of unique IDs in filtered DataFrame: 104
(2118735, 42)


# Get list of available IDs 

[ 80  87  78  92  38 129  95 118  49  94  39  93  79  81  48 119 114  57
 146 113  50  45 106  42 101  74 130  73  99  66  33 125  61  34 122  43
 100  44 107 112  51 115 140  35  60 123  32  98 124 131  72 136  75  46
  41 102 142 117  54 145  53  65  30  62  37 121 134  77  88 133  67 111
 144  52  40 103  47 132 135  36  63 120  31  64 127  91  69  96  29 138
  58  85 139  82  28  97  68  59 108  25  23  24  26  27]

In [27]:
# Get the list of unique IDs in the filtered DataFrame
unique_ids = filtered_df['ID'].unique()

# Print the list of unique IDs
print("List of unique IDs in filtered DataFrame:")
print(unique_ids)
print(len(unique_ids))

List of unique IDs in filtered DataFrame:
[ 80  87  78  92  38 129  95 118  49  94  39  93  79  81  48 119 114  57
 146 113  50  45 106  42 101  74 130  73  99  66  33 125  61  34 122  43
 100  44 107 112  51 115 140  35  60 123  32  98 124 131  72 136  75  46
  41 102 142 117  54 145  53  65  30  62  37 121 134  77  88 133  67 111
 144  52  40 103  47 132 135  36  63 120  31  64 127  91  69  96  29 138
  58  85 139  82  28  97  68  59 108  25  23  24  26  27]
104


# Descriptives frames

In [28]:
# Group by 'ID' and count the number of frames per ID
frame_counts = filtered_df.groupby('ID').size()

# Calculate the minimum, maximum, and average number of frames per ID
min_frames_per_id = frame_counts.min()
max_frames_per_id = frame_counts.max()
average_frames_per_id = frame_counts.mean()

print("Minimum number of frames per ID:", min_frames_per_id)
print("Maximum number of frames per ID:", max_frames_per_id)
print("Average number of frames per ID:", average_frames_per_id)


Minimum number of frames per ID: 8
Maximum number of frames per ID: 37833
Average number of frames per ID: 20372.451923076922


In [29]:
# Group by 'ID' and count the number of frames per ID
frame_counts = filtered_df.groupby('ID').size()

# Convert the frame counts Series to a list and sort it in ascending order
sorted_frame_counts = sorted(frame_counts.tolist())

print("Counts of frames per ID sorted in ascending order:", sorted_frame_counts)


Counts of frames per ID sorted in ascending order: [8, 22, 91, 196, 798, 1172, 1233, 1421, 2068, 9114, 9686, 10702, 14706, 15372, 15429, 15478, 15747, 16382, 16407, 16446, 16562, 16662, 16744, 16872, 17407, 17413, 17458, 17500, 17611, 17666, 17718, 17975, 18239, 18291, 18346, 18432, 18567, 18689, 18703, 19148, 19300, 19544, 19573, 19672, 19857, 19884, 19998, 20219, 20261, 20330, 20695, 21019, 21043, 21189, 21339, 21777, 22020, 22137, 22145, 22376, 22435, 22592, 22879, 22898, 22963, 22997, 23031, 23044, 23099, 23127, 23572, 23807, 23892, 23906, 24287, 24338, 24662, 24744, 24917, 25133, 25270, 25897, 26036, 26107, 27205, 27518, 27661, 27804, 27874, 28201, 28324, 28716, 29140, 29523, 29700, 30361, 31039, 31310, 31749, 31842, 33323, 35252, 35868, 37833]


# Save filtered_df 

In [ ]:
filtered_df.to_csv("/Users/dionnespaltman/Desktop/V5/filtered_df.csv", sep=',')

In [30]:
display(filtered_df)

,Frame,Face_id,Confidence,Success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,ID,Stage,Stage_str
0,104.0,0.0,0.88,1.0,0.00,0.00,0.00,0.00,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
1,105.0,0.0,0.98,1.0,0.00,0.00,0.00,0.00,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
2,112.0,0.0,0.98,1.0,0.31,0.59,0.00,0.00,0.31,0.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
3,115.0,0.0,0.98,1.0,0.00,0.12,0.00,0.00,0.32,0.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
4,154.0,0.0,0.98,1.0,1.26,1.67,0.00,0.00,0.39,0.26,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]","[4, 5, 6]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3106523,21069.0,0.0,0.93,1.0,2.08,2.22,0.89,0.16,0.22,0.47,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,27,"[4, 5, 6]","[4, 5, 6]"
3106524,21070.0,0.0,0.93,1.0,2.03,2.14,0.87,0.69,0.27,0.85,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,27,"[4, 5, 6]","[4, 5, 6]"
3106525,21071.0,0.0,0.93,1.0,2.02,2.11,1.12,0.49,0.47,0.67,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,27,"[4, 5, 6]","[4, 5, 6]"
3106526,21072.0,0.0,0.93,1.0,2.43,2.01,1.11,0.65,0.35,0.82,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,27,"[4, 5, 6]","[4, 5, 6]"
